In [ ]:
#Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import plotly.graph_objs as go
import plotly.subplots as sp
from collections import Counter

In [ ]:
#Reading necessary csvs
meta_df = pd.read_csv("../../aave_data/aave_collateralization_meta_v2.csv")
meta_df.head()

In [ ]:
#Reading necessary csvs
df = pd.read_csv("../../aave_data/aave_collateralization_v2.csv")
df.head()

In [ ]:
#Reading necessary csvs
atokens_df = pd.read_csv("../../aave_data/aave_atokens_v2.csv")
atokens_df.head()

In [ ]:
#Creating a dictionary mapping assets to their decimal values
decimals = { r['symbol']: r['decimals'] for _, r in atokens_df.iterrows() }

In [ ]:
#Adding a date column in df
if 'dt' not in df:
    df['dt'] = df['timestamp'].apply( lambda x: datetime.fromtimestamp(x) )

#Adding a date column in meta_df
if 'dt' not in meta_df:
    meta_df['dt'] = meta_df['timestamp'].apply( lambda x: datetime.fromtimestamp(x) )

In [ ]:
# Check if 'totalDebt' column exists, if not, create it
if 'totalDebt' not in df.columns:
    df.insert(7, 'totalDebt', df['totalStableDebt'].astype(float) + df['totalVariableDebt'].astype(float))
    #df.to_csv("/content/drive/MyDrive/aave_files/aave_collateralization_v2.csv", index = False)


# Check if 'utilizationRate' column exists, if not, create it
if 'utilizationRate' not in df.columns:
    util_rate = df['totalDebt'].astype(float)/(df['availableLiquidity'].astype(float) + df['totalDebt'].astype(float))
    df.insert(8, 'utilizationRate', util_rate)
    #df.to_csv("/content/drive/MyDrive/aave_files/aave_collateralization_v2.csv", index = False)

In [ ]:
#Can include whichever tokens you want in the plots by uncommenting
symbols = [
    'USDC',
    'USDT',
    'WETH',
    # '1INCH',
    # 'AAVE',
    # 'BAL',
    # 'BAT',
    # 'BUSD',
    # 'CRV',
    # 'CVX',
    'DAI',
    # 'DPI',
    # 'ENJ',
    # 'ENS',
    # 'FEI',
    # 'FRAX',
    # 'GUSD',
    # 'KNC',
    # 'LINK',
    # 'MANA',
    # 'MKR',
    # 'RAI',
    # 'REN',
    # 'renFIL',
    # 'SNX',
    # 'stETH',
    # 'sUSD',
    # 'TUSD',
    # 'UNI',
    # 'USDP',
    # 'UST',
    # 'WBTC',
    # 'xSUSHI',
    # 'YFI',
    # 'ZRX'
]


In [ ]:
#Can plot whichever attributes you want in aave_collateralization_v2.csv by uncommenting
target_cols = {
    "availableLiquidity": "Total Value Locked (in USD)",
    "totalVariableDebt": "Total Variable Debt (in USD)",
    "totalStableDebt": "Total Stable Debt (in USD)",
    "totalDebt": "Total Value Borrowed (in USD)",
    "variableBorrowRate": "Variable Borrow Rate (in %)",
    "stableBorrowRate": "Stable Borrow Rate (in %)",
    "liquidityRate": "Liquidity Rate (in %)",
    "utilizationRate": "Utilization Rate (in %)"
}

In [ ]:
#Can plot whichever attributes you want in aave_collateralization_meta_v2.csv by uncommenting
target_cols_meta = {
    "ltv": "LTV (in %)",
    "liquidationThreshold": "Liquidation Threshold (in %)",
    "reserveFactor": "Reserve Factor (in %)",
    "liquidationBonus": "Liquidation Bonus (in %)",
    "isFrozen": "Asset Frozen",
    "isActive": "Asset Active",
    "usageAsCollateralEnabled": "Collateral Enabled",
    "borrowingEnabled": "Borrowing Enabled",
    "stableBorrowEnabled": "Stable Borrow Enabled",
}

In [ ]:
#Function to add USD prices to aave_collateralization_v2.csv
def get_usd():
    df = pd.read_csv("../../aave_data/aave_collateralization_v2.csv")

    symbols = list(df.symbol.unique())

    new_df = pd.DataFrame()

    for symbol in symbols:
        temp_df = pd.read_csv(f"../../aave_data/usd_prices/{symbol}_usd.csv")
        temp_df.astype(dtype={'timestamp':int}, copy=False)
        temp_df.sort_values(by=['timestamp'], inplace=True)
        temp_df['timestamp'] = temp_df['timestamp']//1000
        new_df = pd.concat( [new_df,temp_df] )

    df.sort_values(by=['timestamp'],inplace=True)
    new_df.sort_values(by=['timestamp'],inplace=True)

    df = pd.merge_asof(df, new_df,on = 'timestamp',by = 'symbol', direction = 'nearest')

    df.to_csv("../data/aave_collateralization_v2.csv", index = False)

if 'USD_price' not in df.columns:
  get_usd()

In [ ]:
#Make a pivot table to get usd price for a particular symbol at a particular timetamp
usd_ndf = pd.pivot( df, index=['dt'], columns=['symbol'], values=['USD_price'] )
usd_ndf.columns = [tup[-1] for tup in usd_ndf.columns.to_flat_index()]
usd_ndf.reset_index(inplace=True )
usd_ndf = usd_ndf.astype( { c:float for c in symbols} )

### Plotting Collateralization Meta Plots

#### Plotting data from the aave_collateralization_meta_v2.csv file. This dataset was obtained by invoking the getReserveConfigurationData() function call from the Aave Protocol Data Provider contract, which is part of the Aave V2 ecosystem.

In [ ]:
for target_col, title in target_cols_meta.items():
  # Pivot the meta DataFrame to restructure data for plotting
  meta_ndf = pd.pivot(meta_df, index=['dt'], columns=['symbol'], values=[target_col])
  meta_ndf.columns = [tup[-1] for tup in meta_ndf.columns.to_flat_index()]
  meta_ndf.reset_index(inplace=True)
  meta_ndf = meta_ndf.astype({c: float for c in symbols})

  # Start of Plotly plotting
  fig = go.Figure()

  for symbol in symbols:
      #Note not showing a particular symbol in the plots when its any of these values are 0
      if target_col in ["ltv", "liquidationThreshold", "reserveFactor", "liquidationBonus"]:
          if not (meta_ndf[symbol] == 0).all():
              # Adjusting data for plot
              data = meta_ndf[symbol] // 100
              if target_col == 'liquidationBonus':
                  data = data.apply(lambda x: x - 100 if int(x) != 0 else x)

              fig.add_trace(go.Scatter(x=meta_ndf['dt'], y=data, mode='lines', name=symbol))

      elif target_col in ["isFrozen", "isActive", "usageAsCollateralEnabled", "borrowingEnabled", "stableBorrowEnabled"]:
          fig.add_trace(go.Scatter(x=meta_ndf['dt'], y=meta_ndf[symbol], mode='lines', name=symbol))

  # Adding lines and annotations for significant historical events
  events = [
      {"date": datetime(2022,5,7), "label": "Terra Collapse", "color": "red"},
      {"date": datetime(2022,11,11), "label": "FTX Collapse", "color": "blue"},
      {"date": datetime(2022,9,15), "label": "Ethereum Merge", "color": "orange"}
  ]

  if target_col in ["ltv", "liquidationThreshold", "reserveFactor", "liquidationBonus"]:
    for event in events:
        fig.add_trace(
          go.Scatter(
            x=[event["date"], event["date"]],
            y=[0,100],
            mode='lines',
            line=dict(color=event["color"], width=2),
            name=event["label"],
            showlegend=True,
            hoverinfo='text',
            text = event["label"]
          )
        )

  elif target_col in ["isFrozen", "isActive", "usageAsCollateralEnabled", "borrowingEnabled", "stableBorrowEnabled"]:
    for event in events:
        fig.add_trace(
          go.Scatter(
            x=[event["date"], event["date"]],
            y=[-0.2,1.2],
            mode='lines',
            line=dict(color=event["color"], width=2),
            name=event["label"],
            showlegend=True,
            hoverinfo='text',
            text = event["label"]
          )
        )

  # Update layout
  fig.update_layout(
      title=title,
      xaxis_title='Date',
      yaxis_title=title,
      xaxis=dict(
          rangeselector=dict(
              buttons=list([
                  dict(count=1, label="1m", step="month", stepmode="backward"),
                  dict(count=6, label="6m", step="month", stepmode="backward"),
                  dict(step="all")
              ])
          ),
          type="date"
      )
  )
  fig.show()

### Plotting Collateralization Plots

#### Plotting data from the aave_collateralization_v2.csv file. This dataset was obtained by invoking the getReserveData() function call from the Aave Protocol Data Provider contract, which is part of the Aave V2 ecosystem.

In [ ]:
import plotly.graph_objs as go

for target_col, title in target_cols.items():
  # Data preparation remains the same
  ndf = pd.pivot(df, index=['dt'], columns=['symbol'], values=[target_col])
  ndf.columns = [tup[-1] for tup in ndf.columns.to_flat_index()]
  ndf.reset_index(inplace=True)
  ndf = ndf.astype({c: float for c in symbols})

  # Start of Plotly plotting
  fig = go.Figure()

  if target_col in ["availableLiquidity", "totalVariableDebt", "totalStableDebt", "totalDebt"]:
      for symbol in symbols:
          fig.add_trace(go.Scatter(x=ndf['dt'], y=(ndf[symbol]/(10**decimals[symbol]))*usd_ndf[symbol],
                                    mode='lines', stackgroup='one', name=symbol))

  elif target_col in ["stableBorrowRate", "variableBorrowRate", "liquidityRate", "utilizationRate"]:
      for symbol in symbols:
          rate = ndf[symbol]
          if target_col in ["stableBorrowRate", "variableBorrowRate", "liquidityRate"]:
              rate = rate // (10**25)
          elif target_col == "utilizationRate":
              rate = rate * 100
          fig.add_trace(go.Scatter(x=ndf['dt'], y=rate, mode='lines', name=symbol))

  # Adding lines and annotations for significant historical events
  events = [
      {"date": datetime(2022,5,7), "label": "Terra Collapse", "color": "red"},
      {"date": datetime(2022,11,11), "label": "FTX Collapse", "color": "blue"},
      {"date": datetime(2022,9,15), "label": "Ethereum Merge", "color": "orange"}
  ]

  if target_col in ["availableLiquidity", "totalVariableDebt", "totalStableDebt", "totalDebt"]:
    normalized_values = [(ndf[symbol]/(10**decimals[symbol]))*usd_ndf[symbol] for symbol in symbols]
    y_min = min([values.min() for values in normalized_values])
    y_max = max([values.max() for values in normalized_values])
    for event in events:
      fig.add_trace(
        go.Scatter(
          x=[event["date"], event["date"]],
          y=[y_min, y_max],
          mode='lines',
          line=dict(color=event["color"], width=2),
          name=event["label"],
          showlegend=True,
          hoverinfo='text',
          text = event["label"]
        )
      )

  elif target_col in ["stableBorrowRate", "variableBorrowRate", "liquidityRate", "utilizationRate"]:
    for event in events:
      fig.add_trace(
        go.Scatter(
          x=[event["date"], event["date"]],
          y=[0,100],
          mode='lines',
          line=dict(color=event["color"], width=2),
          name=event["label"],
          showlegend=True,
          hoverinfo='text',
          text = event["label"]
        )
      )

  # Update layout
  fig.update_layout(
      title=title,
      xaxis_title='Date',
      yaxis_title=title,
      xaxis=dict(
          rangeselector=dict(
              buttons=list([
                  dict(count=1, label="1m", step="month", stepmode="backward"),
                  dict(count=6, label="6m", step="month", stepmode="backward"),
                  dict(step="all")
              ])
          ),
          type="date"
      )
  )
  fig.show()